In [52]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer  # 어근추출
from nltk.tokenize import RegexpTokenizer  # 정규표현식을 사용하여 단어 토큰화를 제공
from nltk.corpus import stopwords  # 불용어 정의
import os
import glob
import re

default_path = r'C:\Users\suhyun\Downloads\food_review\tai' # 기본 경로
food_review_total = pd.DataFrame() # 통합할 변수 선언 및 초기화

tokenizer = RegexpTokenizer('[\w]+')  # \w : 단어 영문자+숫자+_(밑줄) [0-9a-zA-Z_]
stop_words = stopwords.words('english')  # 불용어 정의

os.chdir(default_path) 
countries = os.listdir()
os.chdir('review')
food_path = glob.glob('*.csv')
#확장자명을 비교해 csv 리스트들을 만든다.
for path in food_path:
    food_review = pd.read_csv(path, encoding = "UTF-8")
    food_review['label'] = os.path.splitext(os.path.basename(path))[0]
    food_review_total = pd.concat([food_review_total, food_review])

food_review_total = food_review_total.reset_index(drop = True)   
food_review_total = food_review_total[food_review_total['lang'] == 'en']

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

food_review_total['content'] = food_review_total['content'].apply(lambda x:emoji_pattern.sub('', str(x)))
food_review_total = food_review_total[food_review_total['rating'] > 3.0]

temp = food_review_total.groupby(['label'])['content'].apply(lambda x:' '.join(list(x)))

In [53]:
def english_tokenizer_noun_adj(content):
    
    tokenizer = RegexpTokenizer('[\w]+')
    stop_words = stopwords.words('english')
    pos_list = ['NN','NNP','NNS','NNPS','JJ','JJR','JJS']
    word_list = []
    
    words =  str(content).lower()
    tokens = tokenizer.tokenize(words)
    stopped_tokens = [i for i in list(set(tokens)) if not i in stop_words]
    word_pos = nltk.pos_tag(stopped_tokens)
    
    for word,tag in word_pos:
        if tag in pos_list:
            word_list.append(word)
    
    word_list = ' '.join(word_list)
    return word_list

In [54]:
for i in range(len(temp)):
    temp[i] = re.sub('[^a-zA-Z]', ' ',temp[i])
    temp[i] = english_tokenizer_noun_adj(temp[i])
    
temp

label
restaurant_review_2 Fishes Samui                undoubtedly serves windows dent recent familie...
restaurant_review_25 Degrees Bangkok            yummilicious serves fun preference mexican cha...
restaurant_review_3 Spices Restaurant           deliciousnext serves recent cheapest proper or...
restaurant_review_511 Cafe                      families late proper main corner open bags smo...
restaurant_review_57th Street                   ia cups nephew impresse immediate costumer cab...
                                                                      ...                        
restaurant_review_르 노르망디 레스토랑 앳 만다린 오리엔탈, 방콕    gift moments face dent honey immediate pianist...
restaurant_review_시로코 레스토랑                      inc teste moments face immediate cabernet rece...
restaurant_review_시암 수퍼 클럽                      gift undoubtedly moments honey dent cabernet r...
restaurant_review_얌 타이                          lobsters face recent families cheapest late he...
restaurant_rev

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
X = vec.fit_transform(list(temp.values))

df = pd.DataFrame(X.toarray(), columns = vec.get_feature_names(), index = temp.keys())

word_list = []
for i in range(len(temp)):
    temp_df = df.iloc[i].sort_values(ascending=False)[:10]
    print(temp_df)
    for j in temp_df.index.tolist():
        word_list.append(j)

#중복 제거
word_list = list(set(word_list))
print(len(word_list))

#국가에 따라서 경로+파일이름 바꿔주기
f = open(r"C:\Users\suhyun\Downloads\food_review\tai\태국_상위_단어.txt", 'w', -1, "utf-8")
for i in word_list:
    f.write(i+",") #이부분을 , 말고 공백으로 바꿔주면 단어가 ,없이 끊어짐
f.close()

villlage       0.050509
ragua          0.050509
lingiuini      0.050509
pomodorini     0.050509
sighand        0.050509
pomadoro       0.050509
stadig         0.050509
atlantis       0.050509
tagliatelli    0.050509
falvour        0.050509
Name: restaurant_review_2 Fishes Samui, dtype: float64
grn            0.083735
processeco     0.083735
andthey        0.083735
succuclent     0.083735
portebello     0.083735
bannana        0.083735
formulation    0.083735
thinker        0.083735
onuons         0.083735
tainted        0.083735
Name: restaurant_review_25 Degrees Bangkok, dtype: float64
wasdelicious    0.081691
invaded         0.081691
sneeze          0.081691
blonk           0.081691
goodperfect     0.081691
perfecte        0.081691
relaxingwe      0.081691
thaweewong      0.081691
bouno           0.081691
richman         0.081691
Name: restaurant_review_3 Spices Restaurant, dtype: float64
croons          0.089646
lua             0.089646
shortness       0.089646
ensisted        0.089

Name: restaurant_review_Andreas Italian Restaurant & Grill, dtype: float64
nutela                0.075355
embroidered           0.075355
lonley                0.075355
musile                0.075355
crep                  0.075355
muelix                0.075355
mjns                  0.075355
yummmmwe              0.075355
deliciousbreakfast    0.075355
setupfree             0.075355
Name: restaurant_review_Angel's Secrets Cafe, dtype: float64
pongaree        0.078548
etsy            0.078548
dressd          0.078548
creats          0.078548
caroleyeovil    0.078548
pyttipannu      0.078548
reqest          0.078548
guarrera        0.078548
riccardos       0.078548
marko           0.078548
Name: restaurant_review_Aning Restaurant, dtype: float64
anns            0.066117
perfectcome     0.066117
pire            0.066117
pri             0.066117
pirin           0.066117
chargeswould    0.066117
poki            0.066117
pirir           0.066117
wrer            0.066117
perri           0.0661

Name: restaurant_review_Ban Khun Mae Restaurant, dtype: float64
fructe               0.085476
alias                0.085476
treaba               0.085476
engleza              0.085476
salard               0.085476
jevernbe             0.085476
neigborhood          0.085476
daught               0.085476
recommmended         0.085476
makeyourownchoice    0.085476
Name: restaurant_review_Banana Leaf Thai and Indian Restaurant, dtype: float64
custemer     0.087011
coucher      0.087011
cordeon      0.087011
wandering    0.087011
merques      0.087011
taratare     0.087011
nostress     0.087011
grieg        0.087011
abseits      0.087011
transat      0.087011
Name: restaurant_review_Baobab, dtype: float64
youuuuuu     0.048914
promotoin    0.048914
kokkai       0.048914
basu         0.048914
pica         0.048914
sumkumvit    0.048914
nutcha       0.048914
gostel       0.048914
genutzt      0.048914
tastung      0.048914
Name: restaurant_review_BarSu, dtype: float64
barrab       0.065027
ho

Name: restaurant_review_Blue Rice Restaurant by Apple & Noi, dtype: float64
reluef          0.084564
amusant         0.084564
shoalin         0.084564
suagry          0.084564
sustains        0.084564
crispies        0.084564
redicoulous     0.084564
substainable    0.084564
pellogrino      0.084564
plaow           0.084564
Name: restaurant_review_Bo LAN, dtype: float64
skyrail         0.058703
vegeables       0.058703
calisthenics    0.058703
zichar          0.058703
cabbies         0.058703
reststaurant    0.058703
funkytown       0.058703
gaba            0.058703
coursesa        0.058703
sooths          0.058703
Name: restaurant_review_Bo.Lan, dtype: float64
khichri       0.086526
reatsurant    0.086526
fuket         0.086526
adil          0.086526
adilbhai      0.086526
tanvi         0.086526
fatoush       0.086526
sanjeev       0.086526
takeen        0.086526
mannan        0.086526
Name: restaurant_review_Bombay Kitchen & Lounge, dtype: float64
panday        0.05796
menuthank     

Name: restaurant_review_Cabbages and Condoms, dtype: float64
pieyou         0.043091
extrovert      0.043091
tryduck        0.043091
impressedwe    0.043091
ballpark       0.043091
boputh         0.043091
expremental    0.043091
eludes         0.043091
hadbeef        0.043091
makma          0.043091
Name: restaurant_review_Cafe 69, dtype: float64
vegfie                0.045418
breakfastfood         0.045418
februari              0.045418
muffing               0.045418
manydishes            0.045418
coffeine              0.045418
cookmonday            0.045418
lunchwe               0.045418
lovveee               0.045418
thanksregardssabri    0.045418
Name: restaurant_review_Cafe 8.98, dtype: float64
tartatin      0.08604
quiches       0.08604
shouts        0.08604
tarting       0.08604
prochien      0.08604
wirless       0.08604
sainsburys    0.08604
bagutte       0.08604
thjs          0.08604
dolphins      0.08604
Name: restaurant_review_Cafe Tartine, dtype: float64
placen          0.

...

artisans      0.079724
delioclus     0.079724
aow           0.079724
pannecakes    0.079724
shashouska    0.079724
eggsi         0.079724
avotoast      0.079724
slighter      0.079724
cristel       0.079724
brekkies      0.079724
Name: restaurant_review_The Factory Cafe, dtype: float64
correlation            0.073288
magnanimity            0.073288
towpath                0.073288
thevwooden             0.073288
offseason              0.073288
refrigerators          0.073288
gullable               0.073288
usurped                0.073288
twopennybackpackers    0.073288
nicand                 0.073288
Name: restaurant_review_The Family, dtype: float64
theirselves            0.073516
licks                  0.073516
bolls                  0.073516
prtugal                0.073516
connaiseur             0.073516
vegetableswonderful    0.073516
straberry              0.073516
neuter                 0.073516
uhmmm                  0.073516
intented               0.073516
Name: restaurant_revie

Name: restaurant_review_The Never Ending Summer Restaurant, dtype: float64
snot           0.068823
suhi           0.068823
yhoke          0.068823
funnyand       0.068823
yhok           0.068823
familygreat    0.068823
fairthe        0.068823
hellpful       0.068823
karfew         0.068823
grafiew        0.068823
Name: restaurant_review_The Oasis - All Day Dining, dtype: float64
wellsteak      0.082914
vacationthe    0.082914
baggy          0.082914
baggie         0.082914
waigui         0.082914
moktails       0.082914
texturesthe    0.082914
tima           0.082914
satc           0.082914
ttown          0.082914
Name: restaurant_review_The Page, dtype: float64
smoothen          0.065446
todaywell         0.065446
teaux             0.065446
movenpik          0.065446
lotmassaman       0.065446
tryme             0.065446
lenz              0.065446
calientes         0.065446
sinbad            0.065446
proffessionals    0.065446
Name: restaurant_review_The Palm Cuisine, dtype: float64
ho

Name: restaurant_review_Tong Tem Toh, dtype: float64
wather       0.076076
tajmahal     0.076076
viste        0.076076
weekswthe    0.076076
spsey        0.076076
resl         0.076076
ricw         0.076076
riec         0.076076
thear        0.076076
tary         0.076076
Name: restaurant_review_Tourist Taj Mahal, dtype: float64
uruguay       0.054355
pricethere    0.054355
ghoul         0.054355
asmosfair     0.054355
jrakra        0.054355
yoot          0.054355
manuel        0.054355
inthira       0.054355
inging        0.054355
policed       0.054355
Name: restaurant_review_Trader Vic's, dtype: float64
carbonera       0.060169
exelens         0.060169
fegatello       0.060169
gigant          0.060169
sodatotal       0.060169
showpieces      0.060169
etchighly       0.060169
gilrfriend      0.060169
mistocaprese    0.060169
cunsomer        0.060169
Name: restaurant_review_Trattoria Pizzeria Toscana, dtype: float64
ectthey           0.042833
impressivethe     0.042833
cheatworthy    

Name: restaurant_review_Whitening Sairee, dtype: float64
barky             0.071716
oriceyfor         0.071716
slcm              0.071716
durind            0.071716
pakota            0.071716
pubbish           0.071716
parceled          0.071716
arrivedthe        0.071716
occidentals       0.071716
presentationwe    0.071716
Name: restaurant_review_Whole Earth Restaurant, dtype: float64
wineshop      0.090826
centrally     0.090826
winelovers    0.090826
aboutfood     0.090826
selecitons    0.090826
ksm           0.090826
jirapron      0.090826
hoices        0.090826
huffed        0.090826
paso          0.090826
Name: restaurant_review_Wine Connection Deli & Bistro - Central Festival, Koh Samui, dtype: float64
finr               0.080738
jugceylon          0.080738
particularenjoy    0.080738
pegs               0.080738
pricedal           0.080738
rissota            0.080738
taziki             0.080738
neandathol         0.080738
lyrarakis          0.080738
jesters            0.080738


Name: restaurant_review_얌 타이, dtype: float64
thrappaya      0.064303
kachoombar     0.064303
indean         0.064303
pilc           0.064303
zalfrezi       0.064303
undesirable    0.064303
zalfrei        0.064303
insults        0.064303
touristof      0.064303
saucage        0.064303
Name: restaurant_review_인디안 바이 네이처, dtype: float64
7920
